In [10]:
#Importing all the required packages and plugins

import os
import csv
import json
import folium
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster, Search, MeasureControl, FloatImage
from folium import IFrame
from ipywidgets import interact, interactive
import ipywidgets as widget

In [11]:
#Reading tab-delimited/JSON dataset file and storing it in dataframe

df = pd.read_csv('data',delimiter='\t',encoding='utf-8')
df.head()

# data = json.load(open('data.json'))
# df = pd.DataFrame(data["data"])
# df.head()

,I_D,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,EQ_MAG_MB,COUNTRY,LOCATION_NAME,LATITUDE,LONGITUDE,REGION_CODE
0,5503,2000,1,3,22.0,34.0,12.6,33.0,4.6,4.6,INDIA,INDIA-BANGLADESH BORDER: MAHESHKHALI,22.132,92.771,60
1,5504,2000,1,11,23.0,43.0,56.4,10.0,5.1,4.9,CHINA,CHINA: LIAONING PROVINCE,40.498,122.994,30
2,5505,2000,1,14,23.0,37.0,7.8,33.0,5.9,5.4,CHINA,CHINA: YUNNAN PROVINCE: YAOAN COUNTY,25.607,101.063,30
3,5506,2000,2,2,22.0,58.0,1.5,33.0,5.3,5.1,IRAN,"IRAN: BARDASKAN, KASHMAR",35.288,58.218,140
4,5507,2000,2,7,19.0,34.0,57,5.0,4.5,4.5,SOUTH AFRICA,SOUTH AFRICA; SWAZILAND: MBABANE-MANZINI,-26.288,30.888,10


In [12]:
#Selecting specific columns for further operations
#Storing these specific column data in another dataframe

df1 = df[['LATITUDE', 'LONGITUDE', 'LOCATION_NAME', 'EQ_PRIMARY', 'DAY', 'MONTH', 'YEAR']]
df1.head()

,LATITUDE,LONGITUDE,LOCATION_NAME,EQ_PRIMARY,DAY,MONTH,YEAR
0,22.132,92.771,INDIA-BANGLADESH BORDER: MAHESHKHALI,4.6,3,1,2000
1,40.498,122.994,CHINA: LIAONING PROVINCE,5.1,11,1,2000
2,25.607,101.063,CHINA: YUNNAN PROVINCE: YAOAN COUNTY,5.9,14,1,2000
3,35.288,58.218,"IRAN: BARDASKAN, KASHMAR",5.3,2,2,2000
4,-26.288,30.888,SOUTH AFRICA; SWAZILAND: MBABANE-MANZINI,4.5,7,2,2000


In [13]:
#Creating a variable 'locations' which stores the Latitude and Longitude records used to plot the map
#Converting the 'locations' to a list

locations = df1[['LATITUDE', 'LONGITUDE']]
labels = df1[['EQ_PRIMARY', 'LOCATION_NAME', 'DAY','MONTH', 'YEAR']].values.tolist()
locationlist = locations.values.tolist()
len(locationlist)
locationlist[3]

[35.288000000000004, 58.218]

In [14]:
#Function to filter the dataframe with respect to 'year' using an 'interact' slider

def by_year(year):

    locat = df1[['EQ_PRIMARY','LATITUDE','LONGITUDE','LOCATION_NAME','DAY','MONTH','YEAR']]
    
    subset = locat[locat["YEAR"].values == year]
    
    year = subset.merge(locat)
    return year

i = interact(by_year, year=(2000,2018, 1))



interactive(children=(IntSlider(value=2009, description='year', max=2018, min=2000), Output()), _dom_classes=('widget-interact',))

In [15]:
#Finally, selecting the base map layer, feeding the location list dataframe, plotting the data on map

folium_map = folium.Map(location=[11.17, 9.85], zoom_start=2, min_zoom = 2, tiles = "CartoDB dark_matter", max_bounds = True)

#Clustering the nearby Location Markers
marker_cluster = MarkerCluster(control = True).add_to(folium_map)

folium.TileLayer('openstreetmap', min_zoom = 2).add_to(folium_map)
folium.TileLayer('Mapbox Control Room', min_zoom = 2).add_to(folium_map)

#Feature to measure the area of the selected region
folium_map.add_child(MeasureControl())


for point in range(0, len(locationlist)):
    eq_mag = (df1.iloc[point]['EQ_PRIMARY'])

    #Setting text for popup on the markers
    text = ("Magnitude: {}<br> Location: {}<br> Date: {}/{}/{}").format(df1["EQ_PRIMARY"][point],
                                                                        df1["LOCATION_NAME"][point],
                                                                        df1["DAY"][point],
                                                                        df1["MONTH"][point],
                                                                        df1["YEAR"][point])
    
    #Calculating the radius of the Circle Marker based on the Magnitude of Earthquake
    radius = eq_mag * 2
    
    #Setting the colors based on radius
    if radius >= 15:
        color="#FF0000" # red
    elif radius in range(12, 15):
        color="#007849" # green
    elif radius in range(9, 12):
        color="#0A8A9F" # teal
    else:
        color="#FFCE00" # yellow  
    
    #Adding the CircleMarkers for the locations and with the above mentioned features
    folium.CircleMarker(locationlist[point], 
                        radius = radius, 
                        color = color,
                        popup = folium.Popup(IFrame(text, width=230, height=100)),
                        fill=True).add_to(marker_cluster)

folium.LayerControl().add_to(folium_map)
folium_map.save('Exports/map.html')    #Exporting the map in html
folium_map